# Question Answering with LangChain, OpenAI, and MultiQuery Retriever

This interactive workbook demonstrates example of Elasticsearch's [MultiQuery Retriever](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) to generate similar queries for a given user input and apply all queries to retrieve a larger set of relevant documents from a vectorstore.

Before we begin, we first split the fictional workplace documents into passages with `langchain` and uses OpenAI to transform these passages into embeddings and then store these into Elasticsearch.

We will then ask a question, generate similar questions using langchain and OpenAI, retrieve relevant passages from the vector store, and use langchain and OpenAI again to provide a summary for the questions.

## Install packages and import modules

In [1]:
from langchain_community.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain_elasticsearch import ElasticsearchStore
from langchain.retrievers.multi_query import MultiQueryRetriever

print("✅ All LangChain modules imported successfully!")

✅ All LangChain modules imported successfully!


## Connect to Elasticsearch

ℹ️ We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial. 

We'll use the **Cloud ID** to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment.

We will use [ElasticsearchStore](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html) to connect to our elastic cloud deployment, This would help create and index data easily.  We would also send list of documents that we created in the previous step

In [ ]:
OPENAI_API_KEY = 
ELASTIC_CLOUD_ID = 
ELASTIC_API_KEY = 

# Create embedding model
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Connect to Elasticsearch on Elastic Cloud
vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name="multiquery-retriever-index",
    embedding=embeddings,
)

print("✅ Connected to Elastic Cloud successfully!")


C:\Users\macat\AppData\Local\Temp\ipykernel_22868\3664420258.py:9: LangChainPendingDeprecationWarning: The class `ElasticsearchStore` will be deprecated in a future version. Use Use class in langchain-elasticsearch package instead.
  vectorstore = ElasticsearchStore(


✅ Connected to Elastic Cloud successfully!


## Indexing Data into Elasticsearch
Let's download the sample dataset and deserialize the document.

In [4]:
from urllib.request import urlopen
import json

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/example-apps/chatbot-rag-app/data/data.json"

response = urlopen(url)
data = json.load(response)

with open("temp.json", "w") as json_file:
    json.dump(data, json_file)

### Split Documents into Passages

We’ll chunk documents into passages in order to improve the retrieval specificity and to ensure that we can provide multiple passages within the context window of the final question answering prompt.

Here we are chunking documents into 800 token passages with an overlap of 400 tokens.

Here we are using a simple splitter but Langchain offers more advanced splitters to reduce the chance of context being lost.

In [5]:
from langchain_community.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def metadata_func(record: dict, metadata: dict) -> dict:
    # Populate the metadata dictionary if you want to add extra info
    return metadata

loader = JSONLoader(
    file_path="temp.json",
    jq_schema=".[]",
    content_key="content",
    metadata_func=metadata_func,
)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=800,
    chunk_overlap=100,
)
docs = loader.load_and_split(text_splitter=text_splitter)

### Bulk Import Passages

Now that we have split each document into the chunk size of 800, we will now index data to elasticsearch using [ElasticsearchStore.from_documents](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html#langchain.vectorstores.elasticsearch.ElasticsearchStore.from_documents).

We will use Cloud ID, Password and Index name values set in the `Create cloud deployment` step.

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.vectorstores import ElasticsearchStore

# 1️⃣ Create embeddings (using OpenAI)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# 2️⃣ Create your vectorstore (ElasticsearchStore)
vectorstore = ElasticsearchStore.from_documents(
    docs,                      # your list of LangChain Documents
    embedding=embeddings,      # your embedding model
    index_name="my-index",     # name of the index (you can rename)
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
)

# 3️⃣ Initialize the LLM (for MultiQuery)
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

# 4️⃣ Create the MultiQueryRetriever
retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(),
    llm=llm
)


In [13]:
documents = vectorstore.from_documents(
    docs,
    embeddings,
    index_name="multiquery-lab",
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
)

llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)

# Question Answering with MultiQuery Retriever

Now that we have the passages stored in Elasticsearch, we can now ask a question to get the relevant passages.

In [15]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as clear and educational as possible.

Context:
{context}

Question: "{question}"
Answer:
"""
)

# 🔧 Fixed: no `{name}` here anymore
LLM_DOCUMENT_PROMPT = PromptTemplate.from_template(
    """
---
{page_content}
---
"""
)

def _combine_documents(
    docs, document_prompt=LLM_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

_context = RunnableParallel(
    context=retriever | _combine_documents,
    question=RunnablePassthrough(),
)

chain = _context | LLM_CONTEXT_PROMPT | llm

ans = chain.invoke("what is the nasa sales team?")
print("---- Answer ----")
print(ans)


INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the sales team at NASA?', '2. How does the sales team operate within NASA?', '3. What are the responsibilities of the NASA sales team?']


---- Answer ----
The NASA sales team is a part of the Americas region in the sales organization of the company. It is led by two Area Vice-Presidents, Laura Martinez for North America and Gary Johnson for South America. The team is responsible for promoting and selling the company's products and services in the North America and South America regions. They work closely with other departments, such as marketing, product development, and customer support, to ensure the company's success in these regions.


**Generate at least two new iteratioins of the previous cells - Be creative.** Did you master Multi-
Query Retriever concepts through this lab?

In [16]:
questions = [
    "Explain what the NASA sales team is and what they do.",
    "How does the sales team at NASA collaborate with other departments?",
    "What skills would someone need to join the NASA sales team in this dataset?"
]

for q in questions:
    print("\n====================================")
    print("QUESTION:", q)
    print("====================================")
    ans = chain.invoke(q)
    print(ans)



QUESTION: Explain what the NASA sales team is and what they do.


INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide an overview of the NASA sales team and their responsibilities?', '2. What is the purpose of the NASA sales team and how do they contribute to the organization?', '3. Could you elaborate on the role and functions of the NASA sales team?']


The NASA sales team is a part of the Americas region in our company's sales organization. It is led by two Area Vice-Presidents, Laura Martinez for North America and Gary Johnson for South America. The team is responsible for promoting and selling our products and services to customers in the United States, Canada, Mexico, and Central and South America. They work closely with other departments, such as marketing and product development, to identify new business opportunities, nurture existing client relationships, and ensure customer satisfaction. The team also collaborates with the other regional sales teams to achieve the company's overall business objectives.

QUESTION: How does the sales team at NASA collaborate with other departments?


INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the collaboration process like between the sales team at NASA and other departments?', '2. Can you explain the methods used by the sales team at NASA to work together with other departments?', '3. How do different departments within NASA work together with the sales team to achieve their goals?']


The sales team at NASA collaborates with other departments by working closely with them on joint marketing and sales initiatives. They also provide feedback to other departments, such as marketing, product development, and customer support, to ensure that the company consistently delivers high-quality products and services to its clients. Additionally, the sales team may also collaborate with other departments on customer projects or product customizations, providing technical guidance and ensuring that the solutions meet the customer's requirements.

QUESTION: What skills would someone need to join the NASA sales team in this dataset?


INFO:langchain.retrievers.multi_query:Generated queries: ['1. What qualifications are required to become a member of the NASA sales team in this dataset?', '2. What are the necessary skills for joining the NASA sales team as shown in this dataset?', '3. What are the key competencies needed to be a part of the NASA sales team in this dataset?']



To join the NASA sales team, someone would need to have strong communication and collaboration skills, as well as the ability to understand and navigate unique market dynamics and cultural nuances. They should also have experience in identifying and pursuing new business opportunities, nurturing client relationships, and ensuring customer satisfaction. Additionally, knowledge of the sales process and experience in working with other departments, such as marketing and product development, would be beneficial.


In [17]:
q = (
    "I am a new sales hire at NASA. "
    "Summarize the key responsibilities of the NASA sales team in bullet points."
)

ans = chain.invoke(q)

print("=========== Answer for new hire ===========")
print(ans)


INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the main duties of the sales team at NASA?', '2. Can you provide a summary of the key responsibilities for a new sales hire at NASA?', '3. How would you outline the main tasks and duties of the NASA sales team in bullet points?']


=========== Answer for new hire ===========
- Promote and sell company products and services to potential clients in North America and South America
- Establish relationships with customers and understand their needs
- Collaborate with other departments to ensure high-quality products and services are delivered to clients
- Identify and pursue new business opportunities
- Nurture existing client relationships
- Ensure customer satisfaction
- Attend sales meetings and conference calls
- Provide regular product updates and training sessions to the sales team
- Collaborate with engineers on customer projects or product customizations
- Actively seek feedback from the sales team to identify areas of improvement
- Treat colleagues with respect and professionalism
- Collaborate with partners to expand market reach
- Provide customer support and address inquiries and concerns promptly.
